In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

#Assume inputs are human face photos with different angles looking at the phone, some of them with eye closed, some of them with eyes open. 
# Attach label 1 if eyes are open, 0 otherwise.
#First step is to shuffle dataset using specific seed. Then split into training and validation datasets.
# (if need to catch eye gaze, a page of current eyes' focus point needs to be given)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/My Drive/Colab Notebooks/dataset/",
    labels='inferred',
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/My Drive/Colab Notebooks/dataset/",
    labels='inferred',
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

whole_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/My Drive/Colab Notebooks/dataset/",
    labels='inferred',
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
#Second step is to do data augmentation, we do random flip and rotation and changing contrast of the image. 
# We don't do random corp since it adds complexity to detect eyes.
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
    #normalization is also possible                              3)),
    layers.RandomRotation(0.1),
  ]
)
# Then we extract features using convalutional layers, with maxpooling and batch normalization after each layer 
# to extract main features to reduce chance of overfitting.
# If data size is large, underfitting still occurs, 
# at this point we may add extra convalutional layers to extract lower level detail features to better conclude the higher level details of an image
model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.batchNomalization(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.batchNomalization(),
  layers.batchNomalization(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.batchNomalization(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,activation='sigmoid')
])
# Then we check the validation set (once the validation accuracy rate not increase by 1% based on previous accuracy, just stop), 
# then we get the human eyes detection accuracy.

# Further approach is use facial masks 2d matrix (low pass filter, high pass filter) to filter out the face in one image,
# (non facial parts are eliminated) basically eyes are just located in range of faces
# WE detect the range where 0(eliminated value) occurs in the range of face we detected before.
# then we locate the eyes in real image, after extracting eyes, we can consider iris location easily due to different colors from surrounding Sclera. 
# After extracting, we just train on extracted iris with label(tired,
# healthy, and so on) to achieve our main purpose. To catch eye gaze, we need to input pages of current eyes' focus point accordingly (as label) 
# and use regression model to reflect the (x,y) relationship. 

[[0.1 0.1 0.1]
 [0.1 0.1 0.9]
 [0.1 0.9 0.1]
 [0.1 0.9 0.9]
 [0.9 0.1 0.1]
 [0.9 0.1 0.9]
 [0.9 0.9 0.1]
 [0.9 0.9 0.9]]
[[0.1]
 [0.9]
 [0.9]
 [0.9]
 [0.9]
 [0.9]
 [0.9]
 [0.9]]
[[ 0.01373626 -0.15796703 -0.15796703 -0.32967033  0.46703297  0.29532967
   0.29532967  0.12362637]
 [ 0.01373626 -0.15796703  0.46703297  0.29532967 -0.15796703 -0.32967033
   0.29532967  0.12362637]
 [ 0.01373626  0.46703297 -0.15796703  0.29532967 -0.15796703  0.29532967
  -0.32967033  0.12362637]]
[[0.48351648]
 [0.48351648]
 [0.48351648]]
